<a href="https://colab.research.google.com/github/thiagoudweb/Projeto-RAg---1a-VA/blob/main/RAGIPOB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-groq
!pip install streamlit streamlit_chat
!pip install sentence-transformers
!pip install chromadb
!pip install pypdf unstructured
!pip install PyPDF2
!pip install streamlit
!pip install -U langchain langchain-openai





In [ ]:
import langchain
import streamlit as st
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chat_models import ChatOpenAI



In [ ]:
import PyPDF2

def carregar_texto_pdf(caminho_pdf):
    texto_completo = ""
    with open(caminho_pdf, "rb") as arquivo:
        leitor = PyPDF2.PdfReader(arquivo)
        for pagina in range(len(leitor.pages)):
            texto_completo += leitor.pages[pagina].extract_text() + "\n"
    return texto_completo


In [ ]:
from google.colab import files

uploaded = files.upload()  #upload do arquivo manual
caminho_pdf = list(uploaded.keys())[0]

texto_prospecto = carregar_texto_pdf(caminho_pdf)
print(texto_prospecto[:1000])


Saving book.pdf to book (1).pdf
As informações contidas neste Prospecto Preliminar estão sob análise da Comissão de Valores Mobiliários, a qual ainda não se man ifestou a seu respeito. O presente Prospecto Preliminar está sujeito à complementação e correção. O Prospecto Definitivo estará  disponível nas páginas da rede mundial de computadores da Companhia; das
Instituições Participantes da Oferta; das entidades administradoras de mercado organizado de valores mobiliários onde os valore s mobiliários da Companhia sejam admitidos à negociação; e da CVM.  ESTE DOCUMENTO É UMA MINUTA INICIAL SUJEITA A ALTERAÇÕES E COMPLEMENTAÇÕES, TENDO SIDO ARQUIVADO NA COMISSÃO DE VALORES MOBILIÁ RIOS PARA FINS EXCLUSIVOS DE ANÁLISE E EXIGÊNCIAS 
POR PARTE DESSA AUTARQUIA. ESTE DOCUMENTO, PO RTANTO, NÃO SE CARACTERIZA COMO O PROSPECTO PRELIMINAR DA OFERTA E NÃO CONSTITUI U MA OFERTA DE VENDA OU UMA SOLICITAÇÃO PARA 
OFERTA DE COMPRA DE TÍTULOS E VALORES MOBILIÁRIOS NO BRASIL, NOS ESTADOS UNIDOS DA AMÉRIC

In [ ]:
def dividir_texto(texto, tamanho_chunk=500, sobreposicao=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=tamanho_chunk,
        chunk_overlap=sobreposicao,
        separators=["\n\n", "\n", " ", ""]
    )
    return splitter.split_text(texto)




In [ ]:
 chunks = dividir_texto(texto_prospecto)

In [ ]:
# modelo de embeddings usando lib hugging Face
modelo_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# criar embeddings para os chunk de textos
vetores_texto = [modelo_embedding.embed_query(chunk) for chunk in chunks]



<ipython-input-7-4d1219d76d33>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  modelo_embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
# conectando banco para guardar os vetores
chroma_db = Chroma(persist_directory="./chroma_db", embedding_function=modelo_embedding)

# Aadd textos e embeddinss ao banco
ids = [f"chunk_{i}" for i in range(len(chunks))]  # criando identificação para chunk
chroma_db.add_texts(texts=chunks, ids=ids)

print("Base de dados vetorial criada e populada com sucesso!")


Base de dados vetorial criada e populada com sucesso!


In [37]:
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"



# criar o modelo de ia com llama
modelo_groq = ChatGroq(model_name="llama3-8b-8192")



In [38]:
def responder_pergunta_gpt(pergunta, chroma_db, modelo_groq, k=3):
    # Busca os trechos mais relevantes no banco vetorial
    trechos_relevantes = chroma_db.similarity_search(pergunta, k=k)

    # criar o contexto com os embbedings enctrados
    contexto = "\n".join([trecho.page_content for trecho in trechos_relevantes])

    # criar um prompt para o gpt4
    prompt = f"""
    você é um especialista em mercado financeiro e vai usar as informações abaixo para responder a pergunta do usuário com base nos documentos oficiais dos prospectos de ipo.

    CONTEXTO:
    {contexto}

    PERGUNTA:
    {pergunta}

    responda de forma clara,objetiva e baseada apenas no contexto acima.
    """

    # gera a resposta com gpt 4
    resposta = modelo_groq.invoke(prompt)

    return resposta.content  # retornar o texto gerado


In [39]:
pergunta_teste = "auais são os principais riscos mencionados no prospecto desse IPO?"
resposta = responder_pergunta_gpt(pergunta_teste, chroma_db, modelo_groq)

print("Resposta gerada pelo sistema:")
print(resposta)


Resposta gerada pelo sistema:
De acordo com o prospecto, os principais riscos mencionados são:

* Sumário da Companhia – Principais Fatores de Risco Relativos à Companhia (página 17):
	+ Risco de incerteza em relação às expectativas de crescimento e resultado da Companhia;
	+ Risco de perda de competividade;
	+ Risco de mudanças no mercado;
	+ Risco de incapacidade de gerenciar mudanças;
	+ Risco de não alcance de metas;
	+ Risco de perda de talentos;
	+ Risco de alterações na legislação e regulamentação;
* Fatores de Risco Relacionados à Oferta e às Ações (página 64):
	+ Risco de não realização da oferta;
	+ Risco de variação no preço das ações;
	+ Risco de perda de liquidez;
	+ Risco de não alcance de metas da oferta;
* Seção 4. Fatores de Risco (página 288 do Formulário de Referência):
	+ Risco de incerteza em relação às expectativas de crescimento e resultado da Companhia;
	+ Risco de perda de competividade;
	+ Risco de mudanças no mercado;
	+ Risco de incapacidade de gerenciar mud